Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary sentiment polarity labels. It is intended to serve as a benchmark for sentiment classification. This document outlines how the dataset was gathered, and how to use the files provided.

Dataset

The core dataset contains 50,000 reviews split evenly into 25k train and 25k test sets. The overall distribution of labels is balanced (25k pos and 25k neg). We also include an additional 50,000 unlabeled documents for unsupervised learning.

In the entire collection, no more than 30 reviews are allowed for any given movie because reviews for the same movie tend to have correlated ratings. Further, the train and test sets contain a disjoint set of movies, so no significant performance is obtained by memorizing movie-unique terms and their associated with observed labels. In the labeled train/test sets, a negative review has a score <= 4 out of 10, and a positive review has a score >= 7 out of 10. Thus reviews with more neutral ratings are not included in the train/test sets. In the unsupervised set, reviews of any rating are included and there are an even number of reviews > 5 and <= 5.

Files

There are two top-level directories [train/, test/] corresponding to the training and test sets. Each contains [pos/, neg/] directories for the reviews with binary labels positive and negative. Within these directories, reviews are stored in text files named following the convention [[id]_[rating].txt] where [id] is a unique id and [rating] is the star rating for that review on a 1-10 scale. For example, the file [test/pos/200_8.txt] is the text for a positive-labeled test set example with unique id 200 and star rating 8/10 from IMDb. The [train/unsup/] directory has 0 for all ratings because the ratings are omitted for this portion of the dataset.

We also include the IMDb URLs for each review in a separate [urls_[pos, neg, unsup].txt] file. A review with unique id 200 will have its URL on line 200 of this file. Due the ever-changing IMDb, we are unable to link directly to the review, but only to the movie's review page.

In addition to the review text files, we include already-tokenized bag of words (BoW) features that were used in our experiments. These are stored in .feat files in the train/test directories. Each .feat file is in LIBSVM format, an ascii sparse-vector format for labeled data. The feature indices in these files start from 0, and the text tokens corresponding to a feature index is found in [imdb.vocab]. So a line with 0:7 in a .feat file means the first word in [imdb.vocab] (the) appears 7 times in that review.

LIBSVM page for details on .feat file format: http://www.csie.ntu.edu.tw/~cjlin/libsvm/

We also include [imdbEr.txt] which contains the expected rating for each token in [imdb.vocab] as computed by (Potts, 2011). The expected rating is a good way to get a sense for the average polarity of a word in the dataset.

Citing the dataset

When using this dataset please cite our ACL 2011 paper which introduces it. This paper also contains classification results which you may want to compare against.

@InProceedings{maas-EtAl:2011:ACL-HLT2011, author = {Maas, Andrew L. and Daly, Raymond E. and Pham, Peter T. and Huang, Dan and Ng, Andrew Y. and Potts, Christopher}, title = {Learning Word Vectors for Sentiment Analysis}, booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies}, month = {June}, year = {2011}, address = {Portland, Oregon, USA}, publisher = {Association for Computational Linguistics}, pages = {142--150}, url = {http://www.aclweb.org/anthology/P11-1015} }

References

Potts, Christopher. 2011. On the negativity of negation. In Nan Li and David Lutz, eds., Proceedings of Semantics and Linguistic Theory 20, 636-659.

Contact

For questions/comments/corrections please contact Andrew Maas amaas@cs.stanford.edu

    1  Data Import
    2  Data Preprocessing
    3  Feature Engineering
    4  Feature Selection
    5  Model Architecture
    6  Model Training
    7  Model Evaluation
    7.1  Accuracy & Loss
    7.2  Error Analysis
    8  Model Application
    8.1  Test Predictions
    8.2  Custom Reviews

In [1]:
import pandas as pd
import os

In [2]:
path = "./data_movie_reviews/"
positiveFiles = [x for x in os.listdir(path+"train/pos/") if x.endswith(".txt")]
negativeFiles = [x for x in os.listdir(path+"train/neg/") if x.endswith(".txt")]
# testFiles = [x for x in os.listdir(path+"test/") if x.endswith(".txt")]

In [3]:
test_positiveFiles = [x for x in os.listdir(path+"test/pos/") if x.endswith(".txt")]
test_negativeFiles = [x for x in os.listdir(path+"test/neg/") if x.endswith(".txt")]
# test_testFiles = [x for x in os.listdir(path+"test/") if x.endswith(".txt")]

In [4]:
positiveReviews, negativeReviews, testReviews = [], [], []
for pfile in positiveFiles:
    with open(path+"train/pos/"+pfile, encoding="latin1") as f:
        positiveReviews.append(f.read())
for nfile in negativeFiles:
    with open(path+"train/neg/"+nfile, encoding="latin1") as f:
        negativeReviews.append(f.read())
# for tfile in testFiles:
#     with open(path+"test/"+tfile, encoding="latin1") as f:
#         testReviews.append(f.read())

In [5]:
# positiveReviews, negativeReviews, testReviews = [], [], []
# for pfile in positiveFiles:
#     with open(path+"train/pos/"+pfile, encoding="latin1") as f:
#         positiveReviews.append(f.read())
# for nfile in negativeFiles:
#     with open(path+"train/neg/"+nfile, encoding="latin1") as f:
#         negativeReviews.append(f.read())
# for tfile in testFiles:
#     with open(path+"test/"+tfile, encoding="latin1") as f:
#         testReviews.append(f.read())

In [6]:
test_positiveReviews, test_negativeReviews, test_testReviews = [], [], []
for pfile in test_positiveFiles:
    with open(path+"test/pos/"+pfile, encoding="latin1") as f:
        test_positiveReviews.append(f.read())
for nfile in test_negativeFiles:
    with open(path+"test/neg/"+nfile, encoding="latin1") as f:
        test_negativeReviews.append(f.read())
# for tfile in test_testFiles:
#     with open(path+"test/"+tfile, encoding="latin1") as f:
#         testReviews.append(f.read())

In [7]:
# reviews = pd.concat([
#     pd.DataFrame({"review":positiveReviews, "label":1, "file":positiveFiles}),
#     pd.DataFrame({"review":negativeReviews, "label":0, "file":negativeFiles}),
#     pd.DataFrame({"review":testReviews, "label":-1, "file":testFiles})
# ], ignore_index=True).sample(frac=1, random_state=1)
# reviews.head()

In [8]:
reviews = pd.concat([
    pd.DataFrame({"review":positiveReviews, "label":1, "file":positiveFiles}),
    pd.DataFrame({"review":negativeReviews, "label":0, "file":negativeFiles}),
    pd.DataFrame({"review":test_positiveReviews, "label":1, "file":test_positiveFiles}),
    pd.DataFrame({"review":test_negativeReviews, "label":0, "file":test_negativeFiles})
], ignore_index=True).sample(frac=1, random_state=1)
reviews.head()

,file,label,review
26247,11122_8.txt,1,Fame is one of the best movies I've seen about...
35067,7811_10.txt,1,This movie fully deserves to be one of the top...
34590,7382_10.txt,1,"in a time of predictable movies, in which abou..."
16668,2501_1.txt,0,I saw this on TV the other nightÂ or rather I...
12196,9728_7.txt,1,I am a huge fan of Simon Pegg and have watched...


In [9]:
reviews.shape

(50000, 3)

In [10]:
# test_reviews = pd.concat([
#     pd.DataFrame({"review":test_positiveReviews, "label":1, "file":test_positiveFiles}),
#     pd.DataFrame({"review":test_negativeReviews, "label":0, "file":test_negativeFiles})
# ], ignore_index=True).sample(frac=1, random_state=1)
# test_reviews.head()

### Feature engineering

Text preprocessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to feature vectors.

We remove the stop words in order to improve the analytics

In [11]:
from sklearn import feature_extraction
import numpy as np

    print "Creating the bag of words...\n"
    from sklearn.feature_extraction.text import CountVectorizer

    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 


    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()

In [12]:
f = feature_extraction.text.CountVectorizer(stop_words = 'english', ngram_range=(1, 1),max_features=5000 )
X = f.fit_transform(reviews["review"])


In [15]:
np.shape(X)

(50000, 5000)

In [120]:
# f.get_feature_names()

In [16]:
y = reviews['label']

In [17]:
X.shape

(50000, 5000)

In [18]:
y.shape

(50000,)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [21]:
print([np.shape(X_train), np.shape(X_test)])

[(35000, 5000), (15000, 5000)]


In [22]:
y_train.shape, y_test.shape

((35000,), (15000,))

In [23]:
from sklearn.naive_bayes import MultinomialNB 
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
model.predict(X_train)

array([0, 1, 0, ..., 1, 1, 1], dtype=int64)

In [25]:
model.score(X_train, y_train)

0.85228571428571431

In [26]:
model.predict(X_test)

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [27]:
model.score(X_test, y_test)

0.84866666666666668

## Testing Data

In [28]:
from sklearn import metrics

In [29]:
list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = MultinomialNB(alpha=alpha)
    bayes.fit(X_train, y_train)
    score_train[count] = bayes.score(X_train, y_train)
    score_test[count]= bayes.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, bayes.predict(X_test))
    count = count + 1 

In [30]:
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['alpha', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

,alpha,Train Accuracy,Test Accuracy,Test Recall,Test Precision
0,0.00001,0.852171,0.848667,0.843029,0.855441
1,0.11001,0.852143,0.848800,0.843161,0.855575
2,0.22001,0.852143,0.848867,0.843292,0.855594
3,0.33001,0.852086,0.848867,0.843292,0.855594
4,0.44001,0.852086,0.848733,0.843161,0.855460
5,0.55001,0.852171,0.848667,0.842897,0.855536
6,0.66001,0.852200,0.848667,0.842897,0.855536
7,0.77001,0.852257,0.848667,0.842897,0.855536
8,0.88001,0.852286,0.848733,0.843029,0.855556
9,0.99001,0.852286,0.848667,0.842765,0.855631


Lets select the model with the most test precision

In [31]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

alpha             19.910010
Train Accuracy     0.850571
Test Accuracy      0.849533
Test Recall        0.837093
Test Precision     0.861292
Name: 181, dtype: float64

In [32]:
m_confusion_test = metrics.confusion_matrix(y_test, bayes.predict(X_test))
m_confusion_test

array([[6397, 1022],
       [1235, 6346]], dtype=int64)

In [33]:
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,6397,1022
Actual 1,1235,6346
